In [83]:
import pandas as pd

prolongations = pd.read_csv('prolongations.csv')
financial_data = pd.read_csv('financial_data.csv')

financial_data.replace('\xa0', '', regex=True, inplace=True)
financial_data.replace(',', '.', regex=True, inplace=True)

In [84]:
managers = prolongations["AM"].unique()

In [85]:
all_months = ['ноябрь 2022', 'декабрь 2022', 'январь 2023', 'февраль 2023',
              'март 2023', 'апрель 2023', 'май 2023', 'июнь 2023', 'июль 2023',
              'август 2023', 'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023',
              'декабрь 2023', 'январь 2024', 'февраль 2024']

months_to_calc_prolongation = ['январь 2023', 'февраль 2023', 'март 2023', 'апрель 2023',
                  'май 2023', 'июнь 2023', 'июль 2023', 'август 2023',
                  'сентябрь 2023', 'октябрь 2023', 'ноябрь 2023', 'декабрь 2023']

In [86]:
# для начала посчитаем коэффициенты пролонгации для каждого менеджера:
# 1) за каждый месяц (2023 год)
# 2) за год (2023 год)

In [87]:
managers_prolongations = {}

In [88]:
def capitalize_first_letter(string):
    return string[0].upper() + string[1:]

def replace_zeros_with_previous(df):
    for index, row in df.iterrows():
        previous_value = None
        for column in df.columns[1:]:
            if row[column] != 0:
                previous_value = row[column]
            elif row[column] == 0 and previous_value is not None:
                df.at[index, column] = previous_value
    
    return df

def remove_rows_with_zeros(df):
    return df[~df.apply(lambda row: (row == 0).any(), axis=1)]

In [89]:
def delete_stopped_projects(projects_financial_data, prolongation_month_index):
    to_drop = []

    for _, project in projects_financial_data.iterrows():
        for check_month in range(0, prolongation_month_index + 1):
            status = project[capitalize_first_letter(all_months[check_month])]
            if status in ["стоп", "end"]:
                to_drop.append(project.name)
                break

    projects_financial_data = projects_financial_data.drop(index=to_drop, inplace=False)
    return projects_financial_data

def process_financial_data(completed_projects, financial_data, prolongation_month_index):
    ids = completed_projects['id'].unique()
    projects_financial_data = financial_data[financial_data['id'].isin(ids)].copy()
    projects_financial_data = delete_stopped_projects(projects_financial_data, prolongation_month_index)

    required_months = all_months[0: prolongation_month_index + 1]
    required_months = [capitalize_first_letter(month) for month in required_months]
    required_months = ['id'] + required_months

    projects_financial_data = projects_financial_data[required_months]
    projects_financial_data = projects_financial_data.replace("в ноль", 0)
    projects_financial_data = projects_financial_data.replace("NaN", np.nan)
    projects_financial_data = projects_financial_data.apply(pd.to_numeric, errors='coerce')

    projects_financial_data_grouped = projects_financial_data.groupby('id').sum(min_count=1)
    projects_financial_data_grouped = replace_zeros_with_previous(projects_financial_data_grouped)
    projects_financial_data_grouped = remove_rows_with_zeros(projects_financial_data_grouped)

    return projects_financial_data_grouped

In [90]:
# k2 up = сумма отгрузки всех проектов пролонгированных во второй месяц после завершения, например сумма отгрузки тех проектов, завершившихся в марте, у которых нет отгрузки в апреле, но есть в мае (за май).
# k2 down = сумма отгрузки последнего месяца проектов не пролонгированных в первый, например сумма отгрузки проектов, завершившихся в марте, у которых нет отгрузки в апреле (за март)

def calculate_k2_sums(projects_financial_data_grouped, month0, month1, prolongation_month):
    k2_up_sum = 0
    k2_down_sum = 0

    for index, row in projects_financial_data_grouped.iterrows():
        if pd.isna(row[capitalize_first_letter(month1)]) and row[capitalize_first_letter(prolongation_month)] > 0 and row[capitalize_first_letter(month0)] > 0:
            k2_up_sum += row[capitalize_first_letter(prolongation_month)]
        if pd.isna(row[capitalize_first_letter(month1)]) and row[capitalize_first_letter(month0)] > 0:
            k2_down_sum += row[capitalize_first_letter(month0)]

    return k2_up_sum, k2_down_sum

In [91]:
# k1 up = сумма отгрузки всех проектов пролонгированных в первый месяц после завершения, например сумма отгрузки тех проектов завершившихся в апреле, у которых есть отгрузка в мае (за май)
# k1 down = сумма отгрузки последнего месяца реализации всех завершившихся в прошлом месяце проектов, например сумма отгрузки проектов, завершившихся в апреле (за апрель)

def calculate_k1_sums(projects_financial_data_grouped, month1, prolongation_month):
    k1_up_sum = 0
    k1_down_sum = 0

    for index, row in projects_financial_data_grouped.iterrows():
        if row[capitalize_first_letter(month1)] > 0 and row[capitalize_first_letter(prolongation_month)] > 0:
            k1_up_sum += row[capitalize_first_letter(prolongation_month)]
        if row[capitalize_first_letter(month1)] > 0:
            k1_down_sum += row[capitalize_first_letter(month1)]
            
    return k1_up_sum, k1_down_sum

In [92]:
import numpy as np

for manager in managers:
    k1_up_sum_year = 0
    k1_down_sum_year = 0
    k2_up_sum_year = 0
    k2_down_sum_year = 0
    
    for prolongation_month in months_to_calc_prolongation:
        prolongation_month_index = all_months.index(prolongation_month)
        month0_index = all_months.index(prolongation_month) - 2
        month1_index = all_months.index(prolongation_month) - 1
        month0 = all_months[month0_index]
        month1 = all_months[month1_index]

        manager_completed_projects = prolongations[prolongations['AM'] == manager]
        
        month0_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month0]
        month1_completed_projects = manager_completed_projects[manager_completed_projects['month'] == month1]

        
        # СЧИТАЕМ ПРОЕКТЫ, ЗАВЕРШЕННЫЕ В МЕСЯЦЕ 0 (КОЭФФИЦИЕНТ K2)
        projects_financial_data_grouped = process_financial_data(month0_completed_projects, financial_data, prolongation_month_index)
        
        k2_up_sum, k2_down_sum = calculate_k2_sums(projects_financial_data_grouped, month0, month1, prolongation_month)
        
        k2_up_sum_year += k2_up_sum
        k2_down_sum_year += k2_down_sum
        
        if k2_down_sum > 0:
            k2 = k2_up_sum / k2_down_sum
        else:
            k2 = 0
            
        if manager not in managers_prolongations:
            managers_prolongations[manager] = {}
        if prolongation_month not in managers_prolongations[manager]:
            managers_prolongations[manager][prolongation_month] = {}
        managers_prolongations[manager][prolongation_month]['k2'] = k2
        
        
        # СЧИТАЕМ ПРОЕКТЫ, ЗАВЕРШЕННЫЕ В МЕСЯЦЕ 1 (КОЭФФИЦИЕНТ K1)
        projects_financial_data_grouped = process_financial_data(month1_completed_projects, financial_data, prolongation_month_index)
        
        k1_up_sum, k1_down_sum = calculate_k1_sums(projects_financial_data_grouped, month1, prolongation_month)
                
        k1_up_sum_year += k1_up_sum
        k1_down_sum_year += k1_down_sum
                
        if k1_down_sum > 0:
            k1 = k1_up_sum / k1_down_sum
        else:
            k1 = 0
            
        managers_prolongations[manager][prolongation_month]['k1'] = k1
    
    
    # k1 и k2 за год:
    if k2_down_sum_year > 0:
        k2 = k2_up_sum_year / k2_down_sum_year
    else:
        k2 = 0

    if k1_down_sum_year > 0:
        k1 = k1_up_sum_year / k1_down_sum_year
    else:
        k1 = 0

    managers_prolongations[manager]["year"] = {}
    managers_prolongations[manager]["year"]["k1"] = k1
    managers_prolongations[manager]["year"]["k2"] = k2

In [93]:
# мы посчитали коэффициенты пролонгации для каждого менеджера за каждый месяц и за год
# для всего отдела в целом также посчитаем коэффициенты пролонгации за каждый месяц и за год:

department_prolongations = {}

k1_up_sum_year = 0
k1_down_sum_year = 0
k2_up_sum_year = 0
k2_down_sum_year = 0

for prolongation_month in months_to_calc_prolongation:
    prolongation_month_index = all_months.index(prolongation_month)
    month0_index = all_months.index(prolongation_month) - 2
    month1_index = all_months.index(prolongation_month) - 1
    month0 = all_months[month0_index]
    month1 = all_months[month1_index]

    department_completed_projects = prolongations
    
    month0_completed_projects = department_completed_projects[department_completed_projects['month'] == month0]
    month1_completed_projects = department_completed_projects[department_completed_projects['month'] == month1]
    
    
    # СЧИТАЕМ ПРОЕКТЫ, ЗАВЕРШЕННЫЕ В МЕСЯЦЕ 0 (КОЭФФИЦИЕНТ K2)
    projects_financial_data_grouped = process_financial_data(month0_completed_projects, financial_data, prolongation_month_index)
    
    k2_up_sum, k2_down_sum = calculate_k2_sums(projects_financial_data_grouped, month0, month1, prolongation_month)
            
    k2_up_sum_year += k2_up_sum
    k2_down_sum_year += k2_down_sum
    
    if k2_down_sum > 0:
        k2 = k2_up_sum / k2_down_sum
    else:
        k2 = 0
    
    if prolongation_month not in department_prolongations:
        department_prolongations[prolongation_month] = {}
    department_prolongations[prolongation_month]['k2'] = k2
    
    
    # СЧИТАЕМ ПРОЕКТЫ, ЗАВЕРШЕННЫЕ В МЕСЯЦЕ 1 (КОЭФФИЦИЕНТ K1)
    projects_financial_data_grouped = process_financial_data(month1_completed_projects, financial_data, prolongation_month_index)
    
    k1_up_sum, k1_down_sum = calculate_k1_sums(projects_financial_data_grouped, month1, prolongation_month)
            
    k1_up_sum_year += k1_up_sum
    k1_down_sum_year += k1_down_sum
    
    if k1_down_sum > 0:
        k1 = k1_up_sum / k1_down_sum
    else:
        k1 = 0
        
    department_prolongations[prolongation_month]['k1'] = k1
    
# также, за год:
if k2_down_sum_year > 0:
    k2 = k2_up_sum_year / k2_down_sum_year
else:
    k2 = 0
    
if k1_down_sum_year > 0:
    k1 = k1_up_sum_year / k1_down_sum_year
else:
    k1 = 0

if "year" not in department_prolongations:
    department_prolongations["year"] = {}
department_prolongations["year"]["k1"] = k1
department_prolongations["year"]["k2"] = k2

In [94]:
import xlsxwriter
import os

if not os.path.exists('results'):
    os.makedirs('results')
workbook = xlsxwriter.Workbook('results/prolongations_report.xlsx')

managers_sheet_months = workbook.add_worksheet('Managers_months')
managers_sheet_year = workbook.add_worksheet('Managers_year')
department_sheet = workbook.add_worksheet('Department')

managers_sheet_months.write(0, 0, "Менеджер")
managers_sheet_months.write(0, 1, "Месяц")
managers_sheet_months.write(0, 2, "K1")
managers_sheet_months.write(0, 3, "K2")

row = 1
for manager, prolongations in managers_prolongations.items():
    for month, values in prolongations.items():
        if month == "year":
            continue
        managers_sheet_months.write(row, 0, manager)
        managers_sheet_months.write(row, 1, month)
        managers_sheet_months.write(row, 2, values['k1'])
        managers_sheet_months.write(row, 3, values['k2'])
        row += 1
    
managers_sheet_year.write(0, 0, "Менеджер")
managers_sheet_year.write(0, 1, "Год")
managers_sheet_year.write(0, 2, "K1")
managers_sheet_year.write(0, 3, "K2")

row = 1
for manager, prolongations in managers_prolongations.items():
    managers_sheet_year.write(row, 0, manager)
    managers_sheet_year.write(row, 1, "2023 год")
    managers_sheet_year.write(row, 2, prolongations['year']['k1'])
    managers_sheet_year.write(row, 3, prolongations['year']['k2'])
    row += 1
    
department_sheet.write(0, 0, "Месяц")
department_sheet.write(0, 1, "K1")
department_sheet.write(0, 2, "K2")
row = 1
for month, values in department_prolongations.items():
    if month == "year":
        continue
    department_sheet.write(row, 0, month)
    department_sheet.write(row, 1, values['k1'])
    department_sheet.write(row, 2, values['k2'])
    row += 1

department_sheet.write(row, 0, "2023 год")
department_sheet.write(row, 1, department_prolongations['year']['k1'])
department_sheet.write(row, 2, department_prolongations['year']['k2'])
workbook.close()